In [1]:
import numpy as np 
import xarray as xr
import matplotlib.pyplot as plt
from pyresample.geometry import AreaDefinition
from pyresample.geometry import SwathDefinition
from pyresample import kd_tree
from scipy.spatial.distance import euclidean
import pyreadr
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], "..")) # to import from parent directory
from evaluation.metrics import *
from utils import gev2frech
import cmcrameri

# Transform data

This section transforms the raw data from DWD to a lat,lon projection and interpolates the grid to the specified area across Western Germany.

In [2]:
path = "../data/application/data/"

In [4]:
# Define original area
projection = "+proj=lcc +lat_1=35 +lat_2=65 +lat_0=52 +lon_0=10 +x_0=4000000 +y_0=2800000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs"
width = 1200
height = 1100
area_extent = (3500000, 2100000, 4700000, 3200000)
area_def = AreaDefinition('original', 'Original grid', 'id', projection,width, height, area_extent)

# Define target grid
lons = np.linspace(6.38,7.48,30)
lats = np.linspace(50.27,50.97,30)
lon2d, lat2d = np.meshgrid(lons, lats)
target_grid = SwathDefinition(lons = lon2d, lats = lat2d)

In [7]:
# Functions for projection, interpolation and monthly maximum
def downsample(data, area_def, target_grid):
    result = kd_tree.resample_nearest(
        area_def, data, target_grid, radius_of_influence=50000, epsilon=0.5
    )
    return result

def xr_downsample(data, area_def, target_grid):
    result = xr.apply_ufunc(
        downsample,
        data,
        area_def,
        target_grid,
        input_core_dims=[["y", "x", "time"], [], []],
        output_core_dims=[["y", "x", "time"]],
        exclude_dims=set(("x", "y")),
        vectorize = True,
        dask = "allowed"
    )
    return result.max(dim = ["time"])

def transform_data(data, area_def, target_grid, year = ""):
    transformed_data = data.resample(time = "1MS").apply(xr_downsample, args = (area_def, target_grid))
    transformed_data = transformed_data.isel(time=(transformed_data.time.dt.month.isin([6,7,8])))
    xr.Dataset({"pr":transformed_data}).to_netcdf(f"../../data/application/{year}_month_max.nc")

In [6]:
# Training data
raw_data = xr.open_dataset(path + "1931_2020_raw.nc")
data = raw_data.pr.isel(time=(raw_data.time.dt.month.isin([6,7,8])))
transform_data(data, area_def, target_grid, year = "1931_2020")

# 2021
raw_data = xr.open_dataset(path + "2021_raw.nc")
data = raw_data.pr.isel(time=(raw_data.time.dt.month.isin([6,7,8])))
transform_data(data, area_def, target_grid, year = "2021")

# 2022
raw_data = xr.open_dataset(path + "2022_raw.nc")
data = raw_data.pr.isel(time=(raw_data.time.dt.month.isin([6,7,8])))
transform_data(data, area_def, target_grid, year = "2022")

# 2023
raw_data = xr.open_dataset(path + "2023_raw.nc")
data = raw_data.pr.isel(time=(raw_data.time.dt.month.isin([6,7,8])))
transform_data(data, area_def, target_grid, year = "2023")

FileNotFoundError: [Errno 2] No such file or directory: b'c:\\Users\\chris\\Desktop\\Forschung\\MA_paper\\code\\data\\application\\data\\1931_2020_raw.nc'

# Prepare data for CNN

This section reads in the processed data and transforms it to the numpy training/testing data for the neural network.

In [8]:
path = "../data/application/data/"

# Load test data
data_2021 = gev2frech(xr.open_dataset(path + "2021_month_max.nc"), year = 2021)
data_2022 = gev2frech(xr.open_dataset(path + "2022_month_max.nc"), year = 2022)
data_2023 = gev2frech(xr.open_dataset(path + "2023_month_max.nc"), year = 2023)
data = xr.concat([data_2021, data_2022, data_2023], dim = "time")

# Save data
np.save(path + "brown_test_data.npy", data.pr.data)
np.save(path + "powexp_test_data.npy", data.pr.data)
data.to_netcdf(path + "test_data.nc")


# Load predictions

In [3]:
plt.style.use('seaborn-v0_8')
cmap_name = "cmc.roma_r"
cmap = plt.get_cmap(cmap_name)
colors = [cmap(x) for x in np.linspace(0.1,0.99,9)]
labels = ["June 2021", "July 2021", "August 2021", "June 2022", "July 2022", "August 2022", "June 2023", "July 2023", "August 2023"]

In [4]:
model = "powexp"
pl = pyreadr.read_r(f"../data/application/results/{model}_pl.RData")["results"].to_numpy()[:,0:2]
cnn = np.load(f"../data/application/results/{model}_cnn.npy")
cnn_es = np.load(f"../data/application/results/{model}_cnn_es.npy")
cnn_es_mean = cnn_es.mean(axis = 2)

## Save sample predictions

 Save sample predictions of the energy network used for simulating processes.

In [5]:
path = "../data/application/data/"
xr.Dataset({"June":xr.DataArray(cnn_es_mean[3,:]), "July":xr.DataArray(cnn_es_mean[4,:]), "August":xr.DataArray(cnn_es_mean[5,:])}).to_netcdf(path + "parameter_estimates.nc")

## Log score

Calculate the log score across the models and corresponding observations.

In [6]:
def calculate_log_score(data, grid, model, estimation, size = 900):
    # Log score
    n_comb = (size*(size-1))/2
    log_score = 0
    for i in range(data.shape[0]-1):
        for j in range(i+1, data.shape[0]):
            h = euclidean(grid[:,j], grid[:,i])
            density = bivariate_density(data[j], data[i], h, model, estimation[0], estimation[1])
            log_score += -np.log(density)

    log_score = log_score/n_comb
    return log_score

In [8]:
# Load grid
path = "../data/application/data/"
grid = xr.open_dataset(path + "grid.nc").grid.data


# Load test data
grid = xr.open_dataset(path + "grid.nc").grid.data
data_2021 = gev2frech(xr.open_dataset(path + "2021_month_max.nc"), year = 2021)
data_2022 = gev2frech(xr.open_dataset(path + "2022_month_max.nc"), year = 2022)
data_2023 = gev2frech(xr.open_dataset(path + "2023_month_max.nc"), year = 2023)
data = xr.concat([data_2021, data_2022, data_2023], dim = "time")

In [9]:
# Brown
model = "brown"
pl = pyreadr.read_r(f"../data/application/results/{model}_pl.RData")["results"].to_numpy()[:,0:2]
cnn = np.load(f"../data/application/results/{model}_cnn.npy")
cnn_es = np.load(f"../data/application/results/{model}_cnn_es.npy")
cnn_es_mean = cnn_es.mean(axis = 2)


cnn_score = np.zeros(shape = (9))
cnn_es_score = np.zeros(shape = (9))
pl_score = np.zeros(shape = (9))
for i in range(9):
    test = data.isel(time = i).pr.data.flatten()
    cnn_score[i] = calculate_log_score(test, grid, model, cnn[i])
    cnn_es_score[i] = calculate_log_score(test, grid, model, cnn_es_mean[i])
    pl_score[i] = calculate_log_score(test, grid, model, pl[i])
print(f"CNN: {np.round(np.mean(cnn_score), 4)}")
print(f"CNN_ES: {np.round(np.mean(cnn_es_score), 4)}")
print(f"PL: {np.round(np.mean(pl_score), 4)}")

In [ ]:
# Powexp
model = "powexp"
pl = pyreadr.read_r(f"../data/application/results/{model}_pl.RData")["results"].to_numpy()[:,0:2]
cnn = np.load(f"../data/application/results/{model}_cnn.npy")
cnn_es = np.load(f"../data/application/results/{model}_cnn_es.npy")
cnn_es_mean = cnn_es.mean(axis = 2)

cnn_score = np.zeros(shape = (9))
cnn_es_score = np.zeros(shape = (9))
pl_score = np.zeros(shape = (9))
for i in range(9):
    test = data.isel(time = i).pr.data.flatten()
    cnn_score[i] = calculate_log_score(test, grid, model, cnn[i])
    cnn_es_score[i] = calculate_log_score(test, grid, model, cnn_es_mean[i])
    pl_score[i] = calculate_log_score(test, grid, model, pl[i])
print(f"CNN: {np.round(np.mean(cnn_score), 4)}")
print(f"CNN_ES: {np.round(np.mean(cnn_es_score), 4)}")
print(f"PL: {np.round(np.mean(pl_score), 4)}")

C:\Users\chris\Desktop\Masterarbeit\code\evaluation\metrics.py:41: RuntimeWarning: divide by zero encountered in scalar divide
  res = np.exp(-np.power((h / r), s))


CNN: 5.9842
CNN_ES: 5.9792
PL: 6.0078


In [ ]:
# Whitmat
model = "whitmat"
pl = pyreadr.read_r(f"../data/application/results/{model}_pl.RData")["results"].to_numpy()[:,0:2]
cnn = np.load(f"../data/application/results/{model}_cnn.npy")
cnn_es = np.load(f"../data/application/results/{model}_cnn_es.npy")
cnn_es_mean = cnn_es.mean(axis = 2)

cnn_score = np.zeros(shape = (9))
cnn_es_score = np.zeros(shape = (9))
pl_score = np.zeros(shape = (9))
for i in range(9):
    test = data.isel(time = i).pr.data.flatten()
    cnn_score[i] = calculate_log_score(test, grid, model, cnn[i])
    cnn_es_score[i] = calculate_log_score(test, grid, model, cnn_es_mean[i])
    pl_score[i] = calculate_log_score(test, grid, model, pl[i])
print(f"CNN: {np.round(np.mean(cnn_score), 4)}")
print(f"CNN_ES: {np.round(np.mean(cnn_es_score), 4)}")
print(f"PL: {np.round(np.mean(pl_score), 4)}")